<a href="https://colab.research.google.com/github/v4nema/bert_intent_recognition/blob/main/BERT_from_github.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! rm -rf /content/bert_intent_recognition

In [ ]:
! git clone https://github.com/v4nema/bert_intent_recognition.git

import sys
sys.path.insert(1, "/content/bert_intent_recognition")

Cloning into 'bert_intent_recognition'...
remote: Enumerating objects: 38, done.
remote: Counting objects: 100% (38/38), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 38 (delta 16), reused 28 (delta 11), pack-reused 0
Receiving objects: 100% (38/38), 22.52 KiB | 5.63 MiB/s, done.
Resolving deltas: 100% (16/16), done.


In [ ]:
! pip install transformers
! pip install evaluate
! pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 43.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 30.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 107.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 83.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 492.4/492.4 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.8 MB/s eta 0:00:00


In [ ]:
import json
import os
import time
from typing import List

from transformers import AutoTokenizer, pipeline, Pipeline
from datasets import load_dataset, DatasetDict, Dataset
from transformers import TFAutoModelForSequenceClassification
from transformers import create_optimizer
import tensorflow as tf
from transformers.keras_callbacks import KerasMetricCallback
import evaluate

import numpy as np

In [ ]:
from train_intent_recognition import *
from test_intent_recognition import *
import config

config.dataset_files = ['bert_intent_recognition/dataset/part1.tsv', 'bert_intent_recognition/dataset/part2.tsv']
config.train_test_split_path = 'bert_intent_recognition/dataset'
config.results_path = 'bert_intent_recognition/results'
config.num_epochs = 50

In [ ]:
dataset_files = ['bert_intent_recognition/dataset/part1.tsv',
                 'bert_intent_recognition/dataset/part2.tsv']

dataset_dict = load_split_dataset(dataset_files,
                                  'bert_intent_recognition/dataset' if config.save_train_test_split
                                  else None)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Casting to class labels:   0%|          | 0/3098 [00:00<?, ? examples/s]

Creating CSV from Arrow format:   0%|          | 0/3 [00:00<?, ?ba/s]

Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
id2label, label2id = map_id_label(dataset_dict)
len_dataset_dict_train = len(dataset_dict["train"])
model = create_model(id2label, label2id, len_dataset_dict_train)
pipe = train(dataset_dict, model, 'ckpt/best_model')

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertForSequenceClassification: ['bert.embeddings.position_ids']
- This IS expected if you are initializing TFBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias', 'classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
You're using

Epoch 1/50
154/154 [==============================] - 129s 508ms/step - loss: 4.2103 - val_loss: 3.4568 - accuracy: 0.4710
Epoch 2/50
154/154 [==============================] - 57s 373ms/step - loss: 2.9894 - val_loss: 2.3008 - accuracy: 0.6839
Epoch 3/50
154/154 [==============================] - 54s 352ms/step - loss: 2.0361 - val_loss: 1.5935 - accuracy: 0.7871
Epoch 4/50
154/154 [==============================] - 54s 349ms/step - loss: 1.4144 - val_loss: 1.1653 - accuracy: 0.8565
Epoch 5/50
154/154 [==============================] - 48s 313ms/step - loss: 1.0107 - val_loss: 0.8854 - accuracy: 0.8806
Epoch 6/50
154/154 [==============================] - 54s 350ms/step - loss: 0.7372 - val_loss: 0.7193 - accuracy: 0.8903
Epoch 7/50
154/154 [==============================] - 46s 298ms/step - loss: 0.5689 - val_loss: 0.5953 - accuracy: 0.9016
Epoch 8/50
154/154 [==============================] - 19s 122ms/step - loss: 0.4312 - val_loss: 0.5677 - accuracy: 0.8935
Epoch 9/50
154/154 [===

In [ ]:
# def test(dataset: Dataset, pipe: Pipeline, results_path: str):
#     task_evaluator = evaluate.evaluator("text-classification")
#     print('Computing evaluation results... It takes a lot, be patient...')
#     result = task_evaluator.compute(
#         model_or_pipeline=pipe,
#         data=dataset,
#         label_mapping=pipe.model.config.label2id
#     )
#     print('Evaluation completed.')
#     file = os.path.join(results_path, f'{str(int(time.time()))}_evaluation.json')
#     with open(file, "w") as f:
#         json.dump(result, f)
#     return result

In [ ]:
pipe.save_pretrained('bert_intent_recognition/pipe')
results_path = 'bert_intent_recognition/results'

In [ ]:
def test(dataset: Dataset, pipe: Pipeline, results_path: str):
    task_evaluator = evaluator("text-classification")
    print('Computing evaluation results... It takes a lot, be patient...')
    result = task_evaluator.compute(
        model_or_pipeline=pipe,
        data=dataset,
        label_mapping=pipe.model.config.label2id
    )
    print('Evaluation completed.')
    file = os.path.join(results_path, f'{str(int(time.time()))}_evaluation.json')
    with open(file, "w") as f:
        json.dump(result, f)

    return result


def save_scores(dataset: Dataset, pipe: Pipeline, results_path: str):
    res = []
    for text in dataset['text']:
        p_res = pipe(text, top_k=101)
        obj = {
            'text': text,
            'results': p_res
        }
        # obj.update({'results': p_res})
        res.append(obj)

    file = os.path.join(results_path, f'{str(int(time.time()))}_score.json')
    with open(file, "w") as f:
        json.dump(res, f)


In [ ]:
ds_test = Dataset.from_csv(f'{config.train_test_split_path}/test.csv', sep='\t')
result = test(ds_test, pipe, results_path)
save_scores(ds_test, pipe, results_path)
print(result)

Computing evaluation results... It takes a lot, be patient...
Evaluation completed.
{'accuracy': 0.9274193548387096, 'total_time_in_seconds': 260.14336635200016, 'samples_per_second': 2.3833012107680562, 'latency_in_seconds': 0.41958607476129056}


In [ ]:
! zip -r /content/drive/MyDrive/bert_intent_recognition.zip bert_intent_recognition

  adding: bert_intent_recognition/ (stored 0%)
  adding: bert_intent_recognition/ckpt/ (stored 0%)
  adding: bert_intent_recognition/ckpt/best_model.index (deflated 80%)
  adding: bert_intent_recognition/ckpt/.gitkeep (stored 0%)
  adding: bert_intent_recognition/ckpt/checkpoint (deflated 40%)
  adding: bert_intent_recognition/ckpt/best_model.data-00000-of-00001 (deflated 21%)
  adding: bert_intent_recognition/.git/ (stored 0%)
  adding: bert_intent_recognition/.git/index (deflated 50%)
  adding: bert_intent_recognition/.git/description (deflated 14%)
  adding: bert_intent_recognition/.git/config (deflated 34%)
  adding: bert_intent_recognition/.git/HEAD (stored 0%)
  adding: bert_intent_recognition/.git/logs/ (stored 0%)
  adding: bert_intent_recognition/.git/logs/HEAD (deflated 29%)
  adding: bert_intent_recognition/.git/logs/refs/ (stored 0%)
  adding: bert_intent_recognition/.git/logs/refs/heads/ (stored 0%)
  adding: bert_intent_recognition/.git/logs/refs/heads/main (deflated 29%)

In [ ]:
text = "mi spieghi i titoli azionari" #@param {type:"string"}

31 37_share_education_share


3.1205695

In [ ]:
res = []
for text in ds_test['text']:
  p_res = pipe(text, top_k=101)
  obj = {
      'text': text,
      'results': p_res
  }
  # obj.update({'results': p_res})
  res.append(obj)

# score = pipe(ds_test['text'], top_k=101)

In [ ]:
file = os.path.join(results_path, f'{str(int(time.time()))}_score.json')
with open(file, "w") as f:
    json.dump(res, f)